In [1]:
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.8 MB/s eta 0:00:0000:0100:01


In [9]:
import csv
import requests
import psycopg2
import base64
import json
import pandas as pd
import sqlalchemy as db

# Fakturoid API credentials and parameters
FAKTUROID_API_BASE_URL = 'https://app.fakturoid.cz/api/v2/accounts'
FAKTUROID_SLUG = 'datadrivenmarketingdev'
FAKTUROID_USERNAME = 'jan.kadlecek+dev@marketing.bi'
FAKTUROID_API_KEY = '31f47e8632bd4b8e77f3dc7b19c7e2c945c3108b'

file_name = 'expenses'

# Form the complete URL with the slug
FAKTUROID_API_URL = f'{FAKTUROID_API_BASE_URL}/{FAKTUROID_SLUG}/{file_name}.json'

credentials = f"{FAKTUROID_USERNAME}:{FAKTUROID_API_KEY}"
credentials_encoded = base64.b64encode(credentials.encode()).decode('utf-8')
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Basic {credentials_encoded}'
}

# Fetch invoices from Fakturoid API with slug parameter
response = requests.get(FAKTUROID_API_URL, headers=headers)
response.raise_for_status()

# TO DO - check for presence of Link in the response.headers, which indicates pagination.
# print(response.headers)

data = response.json()

# PostgreSQL connection parameters
PG_PARAMS = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres",
    "port": "5432"
}

connection_string = f"postgresql://{PG_PARAMS['user']}:{PG_PARAMS['password']}@{PG_PARAMS['host']}:{PG_PARAMS['port']}/{PG_PARAMS['dbname']}"

df = pd.DataFrame(data)

df.to_csv(f'{file_name}.csv', index=False)
df.to_sql(file_name + '_raw', connection_string, dtype={'lines': sqlalchemy.types.JSON}, if_exists='replace', index=False)

11

In [21]:
print(response.headers)

{'access-control-allow-origin': '*', 'cache-control': 'max-age=0, private, must-revalidate', 'content-encoding': 'gzip', 'content-type': 'application/json; charset=utf-8', 'etag': 'W/"97b81c46ef729b8d2de2647a1874fa31"', 'last-modified': 'Sun, 12 Feb 2023 23:38:43 GMT', 'referrer-policy': 'strict-origin-when-cross-origin', 'set-cookie': 'application_theme=eyJfcmFpbHMiOnsibWVzc2FnZSI6IklteHBaMmgwSWc9PSIsImV4cCI6IjIwMjQtMTItMDZUMjE6NTk6MjAuNjI4WiIsInB1ciI6ImNvb2tpZS5hcHBsaWNhdGlvbl90aGVtZSJ9fQ%3D%3D--6ca14a17ffe08cc9f3326052902a357c362ccaef; path=/; expires=Fri, 06 Dec 2024 21:59:20 GMT; HttpOnly; SameSite=Lax; secure', 'strict-transport-security': 'max-age=63072000; includeSubDomains; preload', 'vary': 'Accept-Encoding', 'x-content-type-options': 'nosniff', 'x-download-options': 'noopen', 'x-frame-options': 'SAMEORIGIN', 'x-permitted-cross-domain-policies': 'none', 'x-request-id': 'e4e24db6-6ad8-4718-84b0-9417fded255b', 'x-runtime': '0.048830', 'x-xss-protection': '0', 'date': 'Wed, 06 D

In [20]:
print(invoices[0].keys())

dict_keys(['id', 'custom_id', 'proforma', 'partial_proforma', 'number', 'number_format_id', 'variable_symbol', 'your_name', 'your_street', 'your_street2', 'your_city', 'your_zip', 'your_country', 'your_registration_no', 'your_vat_no', 'your_local_vat_no', 'client_name', 'client_street', 'client_street2', 'client_city', 'client_zip', 'client_country', 'client_registration_no', 'client_vat_no', 'client_local_vat_no', 'subject_id', 'subject_custom_id', 'generator_id', 'related_id', 'correction', 'correction_id', 'paypal', 'gopay', 'token', 'status', 'order_number', 'issued_on', 'taxable_fulfillment_due', 'due', 'due_on', 'sent_at', 'paid_at', 'reminder_sent_at', 'accepted_at', 'cancelled_at', 'webinvoice_seen_at', 'note', 'footer_note', 'private_note', 'tags', 'bank_account', 'iban', 'swift_bic', 'show_already_paid_note_in_pdf', 'payment_method', 'hide_bank_account', 'currency', 'exchange_rate', 'language', 'transferred_tax_liability', 'eu_electronic_service', 'oss', 'vat_price_mode', 'su

In [27]:
#csv_file = 'invoices.csv'

# Write data to CSV file with quoting and escape character
"""with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file, quoting=csv.QUOTE_ALL, escapechar='\\')
    headers = list(invoices[0].keys())  # Update headers
    csv_writer.writerow(headers)  # Write headers to the CSV file
    for invoice in invoices:
        # Extract values from the invoice dictionary based on the headers order
        data_row = [invoice[header] for header in headers]
        csv_writer.writerow(data_row)  # Write values to the CSV file"""

df = pd.read_csv(csv_file)

# Connect to PostgreSQL
with psycopg2.connect(**PG_PARAMS) as conn:
    # Write DataFrame to PostgreSQL
    df.to_sql('invoices', conn, if_exists='replace', index=False)

/tmp/ipykernel_134/235214578.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('invoices', conn, if_exists='replace', index=False)


DatabaseError: Execution failed on sql '
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type IN ('table', 'view')
            AND name=?;
        ': syntax error at or near ";"
LINE 8:             AND name=?;
                              ^


In [23]:
headers[0]

'id'

In [24]:
query = """
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = 'invoices';"""

# Connect to PostgreSQL
with psycopg2.connect(**PG_PARAMS) as conn:
    with conn.cursor() as cur:
        # Create table dynamically based on JSON keys
        table_creation_query = f"CREATE TABLE IF NOT EXISTS invoices ("
        for idx, header in enumerate(headers):
            # Define column names and types based on JSON keys
            value = invoices[0][header]
            if isinstance(value, (dict, list)): #TO DO: 
                column_type = "JSONB"  # Using JSONB for JSON-like structures
            #elif "id" in header and isinstance(value, int):
            #    column_type = "INTEGER"
            #elif isinstance(value, int) or str(value).isdigit():
            #    column_type = "REAL"  # Assuming other columns as VARCHAR for simplicity
            else:
                column_type = "VARCHAR(255)"  # Assuming other columns as VARCHAR for simplicity
            if idx != 0:
                table_creation_query += ", "
            table_creation_query += f"{header} {column_type}"
        table_creation_query += ");"

        cur.execute(table_creation_query)
        cur.execute(query)
        table_structure = cur.fetchall()

        print("Table Structure:")
        for column in table_structure:
            print(column)

Table Structure:
('lines', 'jsonb')
('tags', 'jsonb')
('eet_records', 'jsonb')
('partial_proforma', 'character varying')
('number', 'character varying')
('number_format_id', 'character varying')
('variable_symbol', 'character varying')
('your_name', 'character varying')
('your_street', 'character varying')
('your_street2', 'character varying')
('your_city', 'character varying')
('your_zip', 'character varying')
('your_country', 'character varying')
('your_registration_no', 'character varying')
('your_vat_no', 'character varying')
('your_local_vat_no', 'character varying')
('client_name', 'character varying')
('client_street', 'character varying')
('client_street2', 'character varying')
('client_city', 'character varying')
('client_zip', 'character varying')
('client_country', 'character varying')
('client_registration_no', 'character varying')
('client_vat_no', 'character varying')
('client_local_vat_no', 'character varying')
('subject_id', 'character varying')
('subject_custom_id', 'c

In [30]:
csv_file = 'output.csv'

# Connect to PostgreSQL
with psycopg2.connect(**PG_PARAMS) as conn:
    with conn.cursor() as cur:
        # Load data into PostgreSQL from CSV
        with open(csv_file, 'r', encoding='utf-8') as f:
            # Skip the header row in the CSV file
            next(f)
            cur.copy_from(f, 'invoices', sep=',', columns=headers, null='None')
            conn.commit()

UndefinedColumn: column "Content-Type" of relation "invoices" does not exist
